In [1]:
import numpy as np
from sklearn.metrics import f1_score

class Layer:
    def __init__(self, input_size=None, output_size=None, weights=None, bias=None):
        self.input = None
        self.output = None
        if (input_size is None or output_size is None) and (weights is None or bias is None):
            raise Exception('input and output sizes or weights with bias are required')
        if weights is None:
            self.weights = np.random.rand(input_size, output_size) # uniform
        else:
            self.weights = weights
            self.input_size = len(self.weights)
            self.output_size = len(self.weights[0])
        if bias is None:
            self.bias = np.random.rand(1, output_size)
        else:
            self.bias = bias

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        d_weights = -learning_rate * weights_error
        d_bias = -learning_rate * output_error
        return input_error, d_weights, d_bias
    
    def update_weights(self, d_weights, d_bias):
        self.weights += d_weights
        self.bias += d_bias
        
    
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.input = None
        self.output = None
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [2]:
def update_list(l, new_el):
    new_l = [None] * len(l)
    for i in range(len(l)-1, 0, -1):
        new_l[i] = l[i-1]
    new_l[0] = new_el
    return new_l

def check_early_stopping(l):
    for i in range(len(l)-1):
        if l[i] < l[i+1]:
            return False
    return True

In [3]:
from copy import deepcopy

class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None
        self.errors = []
        self.val_errors = []

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []

        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate, batch_size, x_val=None,
            y_val=None, early_stopping=None, regularisation=None):
        
        
        if regularisation is not None:
            if type(regularisation) != list:
                raise Exception("Regularisation should be list, example: ['L1', 0.1], where first\
                                element is type of regularisation, second is parameter (lambda)")

        samples = len(x_train)
        
        if early_stopping != None:
            if x_val is None or y_val is None:
                raise Exception('Missing validations sets')
            last_errors = [i for i in range(early_stopping)]
        
        batch_proportion = batch_size/samples

        for i in range(epochs):
            err = 0
            
            for batch_number in range(int(1/batch_proportion)):
                
                layersDeltaWeight = []
                layersDeltaBias = []
                
                for j in range(int(batch_number * batch_size), int((batch_number+1)*batch_size)):
                    
                    #forward propagation
                    
                    output = x_train[j]
                    
                    for layer in self.layers:
                        output = layer.forward_propagation(output)

                    err += self.loss(y_train[j], output)
                    
                    #backward propagation
                    
                    
                    error = self.loss_prime(y_train[j], output)
                    
                    
                    tmplayerDeltaWeights = []
                    tmplayerDeltaBias = []
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, ActivationLayer):
                            
                            error = layer.backward_propagation(error, learning_rate)
                            
                        else:
                            error, deltaWeights, deltaBias = layer.backward_propagation(error, learning_rate)
                            tmplayerDeltaWeights.append(deltaWeights)
                            tmplayerDeltaBias.append(deltaBias)
                        
                    layersDeltaWeight.append(tmplayerDeltaWeights)
                    layersDeltaBias.append(tmplayerDeltaBias)
                    
                for layerdWeight, layerdBias in zip(layersDeltaWeight, layersDeltaBias):
                    
                    only_full_layers = []
                    
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, Layer):
                            
                            only_full_layers.append(layer)
                            
                    for dWeight, dBias, layer in zip(layerdWeight, layerdBias, only_full_layers):
                                
                        layer.update_weights(dWeight, dBias)

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
            
            self.errors.append(err)
            
            if early_stopping != None:
                y_predicted_val = self.predict(x_val)
                new_err = self.loss(y_val, y_predicted_val)

                last_errors = update_list(last_errors, new_err)
                self.val_errors.append(new_err)

                if check_early_stopping(last_errors):
                    print(f'Last errors on validation set: {last_errors}')
                    print('early stopped')
                    break
                    
            if early_stopping is None and x_val is not None:
                
                y_predicted_val = self.predict(x_val)
                new_err = self.loss(y_val, y_predicted_val)
                self.val_errors.append(new_err)
                
            
            
            
    def fit_momentum(self, x_train, y_train, epochs, learning_rate, batch_size, lambd):

        samples = len(x_train)
        
        momentum_w = []
        momentum_b = []
        
        for layer in self.layers:
            
            if isinstance(layer, ActivationLayer):
                continue  
                
            momentum_w.append(deepcopy(layer.weights))
            momentum_b.append(deepcopy(layer.bias))
        
        momentum_w = np.array(momentum_w, dtype=object)
        
        for i in range(len(momentum_w)):
            momentum_w[i][:] = 0
            momentum_b[i][:] = 0

        batch_proportion = batch_size/samples

        for i in range(epochs):
            err = 0
            
            for batch_number in range(int(1/batch_proportion)):
                
                layersDeltaWeight = []
                layersDeltaBias = []
                
                for j in range(int(batch_number * batch_size), int((batch_number+1)*batch_size)):
                    
                    #forward propagation
                    
                    output = x_train[j]
                    
                    for layer in self.layers:
                        output = layer.forward_propagation(output)

                    err += self.loss(y_train[j], output)
                    
                    #backward propagation
                    
                    error = self.loss_prime(y_train[j], output)
                    
                    tmplayerDeltaWeights = []
                    tmplayerDeltaBias = []
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, ActivationLayer):
                            
                            error = layer.backward_propagation(error, learning_rate)
                            
                        else:
                            
                            error, deltaWeights, deltaBias = layer.backward_propagation(error, learning_rate)
                            tmplayerDeltaWeights.append(deltaWeights)
                            tmplayerDeltaBias.append(deltaBias)
                        
                    layersDeltaWeight.append(tmplayerDeltaWeights)
                    layersDeltaBias.append(tmplayerDeltaBias)
                    
                for layerdWeight, layerdBias in zip(layersDeltaWeight, layersDeltaBias):
                    
                    only_full_layers = []
                    
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, Layer):
                            
                            only_full_layers.append(layer)
                            
                    l = len(only_full_layers)
                            
                    for dWeight, dBias, layer in zip(layerdWeight, layerdBias, only_full_layers):
                        
                        l -= 1
                        momentum_w[l] = dWeight + learning_rate * momentum_w[l] * lambd
                        momentum_b[l] = dBias + learning_rate * momentum_b[l] * lambd
                        layer.update_weights(momentum_w[l], momentum_b[l])

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
            self.errors.append(err)
            
    def fit_RMSProp(self, x_train, y_train, epochs, learning_rate, batch_size, beta):
        
        samples = len(x_train)
        
        E_w = []
        E_b = []
        
        for layer in self.layers:
            
            if isinstance(layer, ActivationLayer):
                continue  
                
            E_w.append(deepcopy(layer.weights))
            E_b.append(deepcopy(layer.bias))
        
        E_w = np.array(E_w, dtype=object)
        
        for i in range(len(E_w)):
            E_w[i][:] = 0
            E_b[i][:] = 0
            
        batch_proportion = batch_size/samples

        for i in range(epochs):
            err = 0
            
            for batch_number in range(int(1/batch_proportion)):
                
                layersDeltaWeight = []
                layersDeltaBias = []
                
                for j in range(int(batch_number * batch_size), int((batch_number+1)*batch_size)):
                    
                    #forward propagation
                    
                    output = x_train[j]
                    
                    for layer in self.layers:
                        output = layer.forward_propagation(output)

                    err += self.loss(y_train[j], output)
                    
                    #backward propagation
                    
                    error = self.loss_prime(y_train[j], output)
                    
                    tmplayerDeltaWeights = []
                    tmplayerDeltaBias = []
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, ActivationLayer):
                            
                            error = layer.backward_propagation(error, learning_rate)
                            
                        else:
                            
                            error, deltaWeights, deltaBias = layer.backward_propagation(error, learning_rate)
                            tmplayerDeltaWeights.append(deltaWeights)
                            tmplayerDeltaBias.append(deltaBias)
                        
                    layersDeltaWeight.append(tmplayerDeltaWeights)
                    layersDeltaBias.append(tmplayerDeltaBias)
                    
                for layerdWeight, layerdBias in zip(layersDeltaWeight, layersDeltaBias):
                    
                    only_full_layers = []
                    
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, Layer):
                            
                            only_full_layers.append(layer)
                            
                    l = len(only_full_layers)
                            
                    for dWeight, dBias, layer in zip(layerdWeight, layerdBias, only_full_layers):
                        
                        l -= 1
                        E_w[l] = beta * E_w[l] + (1 - beta) * dWeight ** 2
                        E_b[l] = beta * E_b[l] + (1 - beta) * dBias ** 2
                        layer.update_weights(learning_rate * dWeight / np.sqrt(E_w[l]),
                                             learning_rate * dBias / np.sqrt(E_b[l]))

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
            self.errors.append(err)

In [4]:
def sigmoid(x):
    e = np.exp((-1)*x)
    return 1/(1+e)

def derivative_sigmoid(x):
    e = np.exp((-1)*x)
    return e / ((1+e)**2)

def linear(x):
    return x

def derivative_linear(x):
    return np.array([1 for i in x])

In [5]:
def softmax(x):
    expX = np.exp(x)
    return expX / np.sum(expX)

def derivative_softmax(x):
    s = softmax(x)[0]
    jacobian_m = np.diag(s)
    for i in range(len(jacobian_m)):
        for j in range(len(jacobian_m[0])):
            if i == j:
                jacobian_m[i][j] = 1 - s[i] ** 2
            else:
                jacobian_m[i][j] = -1 * s[i] * s[j]
    return np.diag(jacobian_m)

def tanh(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

def derivative_tanh(x):
    return 1-tanh(x)**2

def relu(x):
    return np.maximum(x, 0)

def derivative_relu(x):
    return np.where(x > 0, 1, 0)

In [6]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [7]:
def cost_function(y_true, y_pred):  # cross-entropy
    
    y = [[0 for i in range(len(net.layers[len(net.layers)-2].output[0]))]]
    y[0][y_true[0][0]] = 1
    y = np.array(y)

    return -np.mean(y * np.log(y_pred + 1e-8))

def cost_function_prime(y_true, y_pred):
    
    y = [[0 for i in range(len(net.layers[len(net.layers)-2].output[0]))]]
    y[0][y_true[0][0]] = 1
    y = np.array(y)
    
    return y_pred - y

# MULTIMODAL-LARGE

In [ ]:
import pandas as pd
data = pd.read_csv('mio1/regression/multimodal-large-training.csv')

In [ ]:
y_train = list(data.y)
y_train = np.array([[[y]] for y in y_train])
x_train = list(data.x)
x_train = np.array([[x] for x in x_train])

In [ ]:
data = pd.read_csv('mio1/regression/multimodal-large-test.csv')
x_test = list(data.x)
y_test = list(data.y)
x_test = np.array([[x] for x in x_test])
y_test = np.array([[[y]] for y in y_test])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=420, test_size=0.2)

## Jedna warstwa ukryta

### SIGMOID

In [ ]:
np.random.seed(20)
net = Network()
net.add(Layer(1, 50))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(50, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.0005, batch_size=50,
        early_stopping=6, x_val=x_val, y_val=y_val)

In [ ]:
err1 = net.errors
out = net.predict(x_test)
mse1 = mse(out, y_test)
print(mse1)

In [ ]:
np.random.seed(20)
net = Network()
net.add(Layer(1, 50))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(50, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=50, learning_rate=0.00001, batch_size=50,
        regularisation=None)

In [ ]:
np.random.seed(20)
net = Network()
net.add(Layer(1, 50))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(50, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=50, learning_rate=0.00001, batch_size=50,
        regularisation=['L1', 0.1])

In [ ]:
np.random.seed(20)
net = Network()
net.add(Layer(1, 50))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(50, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=50, learning_rate=0.00001, batch_size=50,
        regularisation=['L1', 10])

In [ ]:
np.random.seed(20)
net = Network()
net.add(Layer(1, 50))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(50, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=50, learning_rate=0.00001, batch_size=50,
        regularisation=['L1', 1000])

In [ ]:
full_layers_weights = []
for layer in xd:
    if isinstance(layer, Layer):
        full_layers_weights.append(layer.weights)

In [ ]:
for weights in full_layers_weights:
    print(np.sum(np.power(weights, 2)))

### LINIOWA

In [ ]:
np.random.seed(45)
net = Network()
net.add(Layer(1, 50))
net.add(ActivationLayer(linear, derivative_linear))
net.add(Layer(50, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.0001, batch_size=50)

In [ ]:
err2 = net.errors
out = net.predict(x_test)
mse2 = mse(out, y_test)
print(mse2)

### TANH

In [ ]:
np.random.seed(45)
net = Network()
net.add(Layer(1, 50))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(50, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.0002, batch_size=50)

In [ ]:
err3 = net.errors
out = net.predict(x_test)
mse3 = mse(out, y_test)
print(mse3)

### RELU

In [ ]:
np.random.seed(2000)
net = Network()
net.add(Layer(1, 200))
net.add(ActivationLayer(relu, derivative_tanh))
net.add(Layer(200, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.000008, batch_size=10)

In [ ]:
err4 = net.errors
out = net.predict(x_test)
mse4 = mse(out, y_test)
print(mse4)

## Porównanie

In [ ]:
import matplotlib.pyplot as plt

x = list(range(1, 201))

plt.plot(x, err1, label='sigmoid')
plt.plot(x, err2, label='linear')
plt.plot(x, err3, label='tanh')
plt.plot(x, err4, label='ReLU')

plt.legend(loc='upper right')
plt.show()

In [ ]:
print(f'sigmoid_mse: {mse1}\nlinear_mse:{mse2}\ntang_mse: {mse3}\nReLU_mse: {mse4}\n')

## Dwie warstwy ukryte

### SIGMOID

In [ ]:
np.random.seed(15)
net = Network()
net.add(Layer(1, 20))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(20, 30))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(30, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.00013, batch_size=10)

In [ ]:
err1 = net.errors
out = net.predict(x_test)
mse1 = mse(out, y_test)
print(mse1)

### LINIOWA

In [ ]:
np.random.seed(45)
net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(linear, derivative_linear))
net.add(Layer(100, 50))
net.add(ActivationLayer(linear, derivative_linear))
net.add(Layer(50, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.00000001, batch_size=50)

In [ ]:
err2 = net.errors
out = net.predict(x_test)
mse2 = mse(out, y_test)
print(mse2)

### TANH

In [ ]:
np.random.seed(42)
net = Network()
net.add(Layer(1, 10))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(10, 40))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(40, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.0000999, batch_size=50)

In [ ]:
err3 = net.errors
out = net.predict(x_test)
mse3 = mse(out, y_test)
print(mse3)

### RELU

In [ ]:
np.random.seed(10)
net = Network()
net.add(Layer(1, 20))
net.add(ActivationLayer(relu, derivative_relu))
net.add(Layer(20, 10))
net.add(ActivationLayer(relu, derivative_relu))
net.add(Layer(10, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.00001, batch_size=5)

In [ ]:
err4 = net.errors
out = net.predict(x_test)
mse4 = mse(out, y_test)
print(mse4)

## Porównanie

In [ ]:
x = list(range(1, 201))

plt.plot(x, err1, label='sigmoid')
plt.plot(x, err2, label='linear')
plt.plot(x, err3, label='tanh')
plt.plot(x, err4, label='ReLU')

plt.legend(loc='upper right')
plt.ylim([0, 6000])
plt.show()

In [ ]:
print(f'sigmoid_mse: {mse1}\nlinear_mse:{mse2}\ntang_mse: {mse3}\nReLU_mse: {mse4}\n')

## Trzy warstwy ukryte

### SIGMOID

In [ ]:
np.random.seed(10)
net = Network()
net.add(Layer(1, 20))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(20, 20))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(20, 30))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(30, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.0007, batch_size=10)

In [ ]:
err1 = net.errors
out = net.predict(x_test)
mse1 = mse(out, y_test)
print(mse1)

### LINIOWA

In [ ]:
np.random.seed(10)
net = Network()
net.add(Layer(1, 10))
net.add(ActivationLayer(linear, derivative_linear))
net.add(Layer(10, 10))
net.add(ActivationLayer(linear, derivative_linear))
net.add(Layer(10, 10))
net.add(ActivationLayer(linear, derivative_linear))
net.add(Layer(10, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.00001, batch_size=50)

In [ ]:
err2 = net.errors
out = net.predict(x_test)
mse2 = mse(out, y_test)
print(mse2)

### TANH

In [ ]:
np.random.seed(10)
net = Network()
net.add(Layer(1, 5))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(5, 10))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(10, 50))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(50, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.00005, batch_size=50)

In [ ]:
err3 = net.errors
out = net.predict(x_test)
mse3 = mse(out, y_test)
print(mse3)

### RELU

In [ ]:
np.random.seed(10)
net = Network()
net.add(Layer(1, 32))
net.add(ActivationLayer(relu, derivative_relu))
net.add(Layer(32, 16))
net.add(ActivationLayer(relu, derivative_relu))
net.add(Layer(16, 8))
net.add(ActivationLayer(relu, derivative_relu))
net.add(Layer(8, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=200, learning_rate=0.000001, batch_size=5)

In [ ]:
err4 = net.errors
out = net.predict(x_test)
mse4 = mse(out, y_test)
print(mse4)

## Porównanie

In [ ]:
x = list(range(1, 201))

plt.plot(x, err1, label='sigmoid')
plt.plot(x, err2, label='linear')
plt.plot(x, err3, label='tanh')
plt.plot(x, err4, label='ReLU')

plt.legend(loc='upper right')
plt.show()

In [ ]:
print(f'sigmoid_mse: {mse1}\nlinear_mse:{mse2}\ntang_mse: {mse3}\nReLU_mse: {mse4}\n')

# steps-large

In [ ]:
data = pd.read_csv('mio1/regression/steps-large-training.csv')
y_train = list(data.y)
y_train = np.array([[[y]] for y in y_train])
x_train = list(data.x)
x_train = np.array([[x] for x in x_train])

In [ ]:
data = pd.read_csv('mio1/regression/steps-large-test.csv')
y_test = list(data.y)
y_test = np.array([[[y]] for y in y_test])
x_test = list(data.x)
x_test = np.array([[x] for x in x_test])

In [ ]:
np.random.seed(20)
net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(100, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=100, learning_rate=0.0001, batch_size=50)

In [ ]:
err1 = net.errors
out = net.predict(x_test)
mse1 = mse(out, y_test)
print(mse1)

In [ ]:
np.random.seed(20)
net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(100, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=100, learning_rate=0.0001, batch_size=50)

In [ ]:
err2 = net.errors
out = net.predict(x_test)
mse2 = mse(out, y_test)
print(mse2)

In [ ]:
x = list(range(1, 101))

plt.plot(x, err1, label='tanh')
plt.plot(x, err2, label='sigmoid')

plt.legend(loc='upper right')
plt.show()

# rings3-regular

In [ ]:
data = pd.read_csv('mio1/classification/rings3-regular-training.csv')
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

x_train = data[['x', 'y']].values.tolist()
x_train = np.array([[x] for x in x_train])
y_train = list(data.c)
y_train = np.array([[[y]] for y in y_train])

data = pd.read_csv('mio1/classification/rings3-regular-test.csv')
x_test = data[['x', 'y']].values.tolist()
x_test = np.array([[x] for x in x_test])
y_test = list(data.c)

In [ ]:
np.random.seed(15)
net = Network()
net.add(Layer(2, 20))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(20, 30))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(30, 3))
net.add(ActivationLayer(softmax, derivative_softmax))

net.use(cost_function, cost_function_prime)
net.fit(x_train, y_train, epochs=3000, learning_rate=0.00013, batch_size=10)

In [ ]:
err_sigmoid = net.errors
out = net.predict(x_test)
out = [np.argmax(l) for l in out]
f1_score(y_test, out, average = 'weighted')

In [ ]:
np.random.seed(52)
net = Network()
net.add(Layer(2, 50))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(50, 10))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(10, 3))
net.add(ActivationLayer(softmax, derivative_softmax))

net.use(cost_function, cost_function_prime)
net.fit(x_train, y_train, epochs=3000, learning_rate=0.00006, batch_size=250)

In [ ]:
err_tanh = net.errors
out = net.predict(x_test)
out = [np.argmax(l) for l in out]
f1_score(y_test, out, average = 'weighted')

In [ ]:
x = list(range(1, 3001))

plt.plot(x, err_tanh, label='tanh')
plt.plot(x, err_sigmoid, label='sigmoid')

plt.legend(loc='upper right')
plt.show()

# rings5-regular

In [ ]:
data = pd.read_csv('mio1/classification/rings5-regular-training.csv')
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

x_train = data[['x', 'y']].values.tolist()
x_train = np.array([[x] for x in x_train])
y_train = list(data.c)
y_train = np.array([[[y]] for y in y_train])

data = pd.read_csv('mio1/classification/rings5-regular-test.csv')
x_test = data[['x', 'y']].values.tolist()
x_test = np.array([[x] for x in x_test])
y_test = list(data.c)

In [ ]:
np.random.seed(15)
net = Network()
net.add(Layer(2, 20))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(20, 30))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(30, 5))
net.add(ActivationLayer(softmax, derivative_softmax))

net.use(cost_function, cost_function_prime)
net.fit(x_train, y_train, epochs=3000, learning_rate=0.00013, batch_size=10)

In [ ]:
err_sigmoid = net.errors
out = net.predict(x_test)
out = [np.argmax(l) for l in out]
f1_score(y_test, out, average = 'weighted')

In [ ]:
np.random.seed(52)
net = Network()
net.add(Layer(2, 50))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(50, 10))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(10, 5))
net.add(ActivationLayer(softmax, derivative_softmax))

net.use(cost_function, cost_function_prime)
net.fit(x_train, y_train, epochs=3000, learning_rate=0.00006, batch_size=250)

In [ ]:
err_tanh = net.errors
out = net.predict(x_test)
out = [np.argmax(l) for l in out]
f1_score(y_test, out, average = 'weighted')

In [ ]:
x = list(range(1, 3001))

plt.plot(x, err_tanh, label='tanh')
plt.plot(x, err_sigmoid, label='sigmoid')

plt.legend(loc='upper right')
plt.show()

In [10]:
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv('mio1/regression/multimodal-sparse-training.csv')
y_train = list(data.y)
y_train = np.array([[[y]] for y in y_train])
x_train = list(data.x)
x_train = np.array([[x] for x in x_train])

data = pd.read_csv('mio1/regression/multimodal-sparse-test.csv')
x_test = list(data.x)
y_test = list(data.y)
x_test = np.array([[x] for x in x_test])
y_test = np.array([[[y]] for y in y_test])

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=420, test_size=0.2)

In [9]:
np.random.seed(20)
net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(100, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=10000, learning_rate=0.0005, batch_size=20, x_val=x_val, y_val=y_val)

NameError: name 'x_train' is not defined

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

err1 = net.errors
err2 = net.val_errors

x = list(range(1, len(err1)+1))

plt.plot(x, err1, label='train')
plt.plot(x, err2, label='val')

plt.legend(loc='upper right')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.show()

In [ ]:
out = net.predict(x_test)
mse = mse(out, y_test)
print(mse) # 689.2154091460627

In [13]:
np.random.seed(20)
net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(tanh, derivative_tanh))
net.add(Layer(100, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=10000, learning_rate=0.0005, batch_size=20,
        early_stopping=4, x_val=x_val, y_val=y_val)

epoch 1/10000   error=3741.545857
epoch 2/10000   error=3547.755547
epoch 3/10000   error=3415.371246
epoch 4/10000   error=3281.065919
epoch 5/10000   error=3152.714342
epoch 6/10000   error=3032.426333
epoch 7/10000   error=2917.271566
epoch 8/10000   error=2805.799476
epoch 9/10000   error=2698.790024
epoch 10/10000   error=2598.217397
epoch 11/10000   error=2505.815290
epoch 12/10000   error=2421.647612
epoch 13/10000   error=2344.358053
epoch 14/10000   error=2272.568469
epoch 15/10000   error=2205.511556
epoch 16/10000   error=2142.881111
epoch 17/10000   error=2084.448676
epoch 18/10000   error=2029.943203
epoch 19/10000   error=1979.084074
epoch 20/10000   error=1931.587316
epoch 21/10000   error=1887.164056
epoch 22/10000   error=1845.525284
epoch 23/10000   error=1806.389867
epoch 24/10000   error=1769.494939
epoch 25/10000   error=1734.606817
epoch 26/10000   error=1701.530676
epoch 27/10000   error=1670.115909
epoch 28/10000   error=1640.253830
epoch 29/10000   error=1611.8

epoch 250/10000   error=71.315238
epoch 251/10000   error=69.763596
epoch 252/10000   error=68.267637
epoch 253/10000   error=66.826207
epoch 254/10000   error=65.436331
epoch 255/10000   error=64.096689
epoch 256/10000   error=62.804729
epoch 257/10000   error=61.559049
epoch 258/10000   error=60.357433
epoch 259/10000   error=59.198456
epoch 260/10000   error=58.080174
epoch 261/10000   error=57.001179
epoch 262/10000   error=55.959747
epoch 263/10000   error=54.954517
epoch 264/10000   error=53.983946
epoch 265/10000   error=53.046737
epoch 266/10000   error=52.141500
epoch 267/10000   error=51.267009
epoch 268/10000   error=50.422010
epoch 269/10000   error=49.605352
epoch 270/10000   error=48.815895
epoch 271/10000   error=48.052567
epoch 272/10000   error=47.314330
epoch 273/10000   error=46.600190
epoch 274/10000   error=45.909197
epoch 275/10000   error=45.240432
epoch 276/10000   error=44.593028
epoch 277/10000   error=43.966136
epoch 278/10000   error=43.358962
epoch 279/1000

epoch 500/10000   error=21.670466
epoch 501/10000   error=24.636014
epoch 502/10000   error=21.560445
epoch 503/10000   error=24.526225
epoch 504/10000   error=21.453585
epoch 505/10000   error=24.419933
epoch 506/10000   error=21.349814
epoch 507/10000   error=24.317029
epoch 508/10000   error=21.249056
epoch 509/10000   error=24.217403
epoch 510/10000   error=21.151228
epoch 511/10000   error=24.120943
epoch 512/10000   error=21.056243
epoch 513/10000   error=24.027532
epoch 514/10000   error=20.964014
epoch 515/10000   error=23.937056
epoch 516/10000   error=20.874450
epoch 517/10000   error=23.849399
epoch 518/10000   error=20.787459
epoch 519/10000   error=23.764447
epoch 520/10000   error=20.702951
epoch 521/10000   error=23.682087
epoch 522/10000   error=20.620832
epoch 523/10000   error=23.602208
epoch 524/10000   error=20.541014
epoch 525/10000   error=23.524702
epoch 526/10000   error=20.463407
epoch 527/10000   error=23.449464
epoch 528/10000   error=20.387922
epoch 529/1000

epoch 740/10000   error=16.709946
epoch 741/10000   error=19.727387
epoch 742/10000   error=16.689678
epoch 743/10000   error=19.705277
epoch 744/10000   error=16.669510
epoch 745/10000   error=19.683248
epoch 746/10000   error=16.649441
epoch 747/10000   error=19.661298
epoch 748/10000   error=16.629470
epoch 749/10000   error=19.639426
epoch 750/10000   error=16.609595
epoch 751/10000   error=19.617632
epoch 752/10000   error=16.589816
epoch 753/10000   error=19.595916
epoch 754/10000   error=16.570132
epoch 755/10000   error=19.574277
epoch 756/10000   error=16.550541
epoch 757/10000   error=19.552715
epoch 758/10000   error=16.531044
epoch 759/10000   error=19.531228
epoch 760/10000   error=16.511639
epoch 761/10000   error=19.509818
epoch 762/10000   error=16.492325
epoch 763/10000   error=19.488483
epoch 764/10000   error=16.473101
epoch 765/10000   error=19.467223
epoch 766/10000   error=16.453967
epoch 767/10000   error=19.446038
epoch 768/10000   error=16.434922
epoch 769/1000

epoch 1007/10000   error=17.366251
epoch 1008/10000   error=14.649604
epoch 1009/10000   error=17.352147
epoch 1010/10000   error=14.637973
epoch 1011/10000   error=17.338086
epoch 1012/10000   error=14.626383
epoch 1013/10000   error=17.324069
epoch 1014/10000   error=14.614835
epoch 1015/10000   error=17.310093
epoch 1016/10000   error=14.603327
epoch 1017/10000   error=17.296161
epoch 1018/10000   error=14.591860
epoch 1019/10000   error=17.282271
epoch 1020/10000   error=14.580434
epoch 1021/10000   error=17.268423
epoch 1022/10000   error=14.569047
epoch 1023/10000   error=17.254617
epoch 1024/10000   error=14.557701
epoch 1025/10000   error=17.240852
epoch 1026/10000   error=14.546394
epoch 1027/10000   error=17.227129
epoch 1028/10000   error=14.535126
epoch 1029/10000   error=17.213448
epoch 1030/10000   error=14.523898
epoch 1031/10000   error=17.199807
epoch 1032/10000   error=14.512709
epoch 1033/10000   error=17.186207
epoch 1034/10000   error=14.501558
epoch 1035/10000   e

...

epoch 8933/10000   error=5.309874
epoch 8934/10000   error=5.330475
epoch 8935/10000   error=5.309088
epoch 8936/10000   error=5.329778
epoch 8937/10000   error=5.308302
epoch 8938/10000   error=5.329080
epoch 8939/10000   error=5.307515
epoch 8940/10000   error=5.328383
epoch 8941/10000   error=5.306728
epoch 8942/10000   error=5.327685
epoch 8943/10000   error=5.305942
epoch 8944/10000   error=5.326987
epoch 8945/10000   error=5.305155
epoch 8946/10000   error=5.326289
epoch 8947/10000   error=5.304368
epoch 8948/10000   error=5.325590
epoch 8949/10000   error=5.303581
epoch 8950/10000   error=5.324892
epoch 8951/10000   error=5.302793
epoch 8952/10000   error=5.324193
epoch 8953/10000   error=5.302006
epoch 8954/10000   error=5.323495
epoch 8955/10000   error=5.301219
epoch 8956/10000   error=5.322796
epoch 8957/10000   error=5.300431
epoch 8958/10000   error=5.322096
epoch 8959/10000   error=5.299643
epoch 8960/10000   error=5.321397
epoch 8961/10000   error=5.298855
epoch 8962/100

epoch 9173/10000   error=5.214388
epoch 9174/10000   error=5.245294
epoch 9175/10000   error=5.213579
epoch 9176/10000   error=5.244568
epoch 9177/10000   error=5.212770
epoch 9178/10000   error=5.243842
epoch 9179/10000   error=5.211961
epoch 9180/10000   error=5.243116
epoch 9181/10000   error=5.211151
epoch 9182/10000   error=5.242389
epoch 9183/10000   error=5.210341
epoch 9184/10000   error=5.241663
epoch 9185/10000   error=5.209531
epoch 9186/10000   error=5.240935
epoch 9187/10000   error=5.208721
epoch 9188/10000   error=5.240208
epoch 9189/10000   error=5.207910
epoch 9190/10000   error=5.239480
epoch 9191/10000   error=5.207099
epoch 9192/10000   error=5.238752
epoch 9193/10000   error=5.206288
epoch 9194/10000   error=5.238023
epoch 9195/10000   error=5.205476
epoch 9196/10000   error=5.237295
epoch 9197/10000   error=5.204664
epoch 9198/10000   error=5.236565
epoch 9199/10000   error=5.203852
epoch 9200/10000   error=5.235836
epoch 9201/10000   error=5.203039
epoch 9202/100

epoch 9440/10000   error=5.145344
epoch 9441/10000   error=5.102750
epoch 9442/10000   error=5.144561
epoch 9443/10000   error=5.101887
epoch 9444/10000   error=5.143778
epoch 9445/10000   error=5.101022
epoch 9446/10000   error=5.142993
epoch 9447/10000   error=5.100158
epoch 9448/10000   error=5.142209
epoch 9449/10000   error=5.099292
epoch 9450/10000   error=5.141423
epoch 9451/10000   error=5.098426
epoch 9452/10000   error=5.140638
epoch 9453/10000   error=5.097560
epoch 9454/10000   error=5.139851
epoch 9455/10000   error=5.096693
epoch 9456/10000   error=5.139064
epoch 9457/10000   error=5.095826
epoch 9458/10000   error=5.138277
epoch 9459/10000   error=5.094958
epoch 9460/10000   error=5.137489
epoch 9461/10000   error=5.094089
epoch 9462/10000   error=5.136700
epoch 9463/10000   error=5.093220
epoch 9464/10000   error=5.135911
epoch 9465/10000   error=5.092350
epoch 9466/10000   error=5.135121
epoch 9467/10000   error=5.091480
epoch 9468/10000   error=5.134331
epoch 9469/100

epoch 9697/10000   error=4.987667
epoch 9698/10000   error=5.039573
epoch 9699/10000   error=4.986734
epoch 9700/10000   error=5.038716
epoch 9701/10000   error=4.985800
epoch 9702/10000   error=5.037859
epoch 9703/10000   error=4.984866
epoch 9704/10000   error=5.037002
epoch 9705/10000   error=4.983931
epoch 9706/10000   error=5.036144
epoch 9707/10000   error=4.982996
epoch 9708/10000   error=5.035286
epoch 9709/10000   error=4.982061
epoch 9710/10000   error=5.034427
epoch 9711/10000   error=4.981125
epoch 9712/10000   error=5.033567
epoch 9713/10000   error=4.980189
epoch 9714/10000   error=5.032707
epoch 9715/10000   error=4.979252
epoch 9716/10000   error=5.031847
epoch 9717/10000   error=4.978315
epoch 9718/10000   error=5.030986
epoch 9719/10000   error=4.977377
epoch 9720/10000   error=5.030125
epoch 9721/10000   error=4.976439
epoch 9722/10000   error=5.029263
epoch 9723/10000   error=4.975501
epoch 9724/10000   error=5.028401
epoch 9725/10000   error=4.974562
epoch 9726/100

epoch 9946/10000   error=4.930948
epoch 9947/10000   error=4.869180
epoch 9948/10000   error=4.930066
epoch 9949/10000   error=4.868233
epoch 9950/10000   error=4.929184
epoch 9951/10000   error=4.867286
epoch 9952/10000   error=4.928302
epoch 9953/10000   error=4.866340
epoch 9954/10000   error=4.927420
epoch 9955/10000   error=4.865394
epoch 9956/10000   error=4.926538
epoch 9957/10000   error=4.864448
epoch 9958/10000   error=4.925657
epoch 9959/10000   error=4.863503
epoch 9960/10000   error=4.924776
epoch 9961/10000   error=4.862558
epoch 9962/10000   error=4.923894
epoch 9963/10000   error=4.861613
epoch 9964/10000   error=4.923014
epoch 9965/10000   error=4.860669
epoch 9966/10000   error=4.922133
epoch 9967/10000   error=4.859725
epoch 9968/10000   error=4.921252
epoch 9969/10000   error=4.858781
epoch 9970/10000   error=4.920372
epoch 9971/10000   error=4.857838
epoch 9972/10000   error=4.919492
epoch 9973/10000   error=4.856895
epoch 9974/10000   error=4.918612
epoch 9975/100

In [ ]:
err1 = net.errors
err2 = net.val_errors

x = list(range(1, len(err1)+1))

plt.plot(x, err1, label='train')
plt.plot(x, err2, label='val')

plt.legend(loc='upper right')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.show()